### Authenticate and get headers

In [ ]:
import requests
from _includes import config
from _includes import helper
import json
import pandas as pd

config_dict = config.config_dict

api = helper.APIHelper(config_dict)
base_site_url = f'{api.server}/api/{api.api_version}/sites/{api.site_id}'
base_api_url = f'{api.server}/api/-'
headers = api.headers

### List all of metric definitions (with Helper Class)

In [ ]:
definition_dict = api.get_pulse_metric_definitions()
definition_dict = {key: value for key, value in definition_dict.items() if 'Number of Visitor' in key}
definition_df = pd.DataFrame(list(definition_dict.items()), columns=['definition_name', 'definition_id'])
definition_df

### Pickup the definition

In [ ]:
definition_id = definition_dict.get('Number of Visitor 01')
definition_id

In [ ]:
# GRANULARITY_BY_YEAR / GRANULARITY_BY_QUARTER / GRANULARITY_BY_MONTH / GRANULARITY_BY_WEEK / GRANULARITY_BY_DAY
granularity = 'GRANULARITY_BY_QUARTER'
# RANGE_CURRENT_PARTIAL / RANGE_LAST_COMPLETE
range = 'RANGE_LAST_COMPLETE'
# TIME_COMPARISON_PREVIOUS_PERIOD / TIME_COMPARISON_YEAR_AGO_PERIOD
time_comparison = 'TIME_COMPARISON_YEAR_AGO_PERIOD'

In [ ]:
payload = {
    'definition_id': definition_id,
    'specification': {
        'filters': [],
        'measurement_period': {
            'granularity': granularity,
            'range': range
        },
        'time_comparison' : time_comparison
    }
}

### Create the metric

In [ ]:
try:
    endpoint = f'{base_api_url}/pulse/metrics'
    api_response = requests.post(endpoint, headers=headers, json=payload)
    api_response.raise_for_status()
    api_response_json = api_response.json()
    metric_id = api_response_json.get('metric').get('id')    
    print('Metric definition created successfully.')
    print('Metric ID:')
    print(metric_id)
except requests.exceptions.RequestException as e:
    print(f'Request failed: {e}')
    if api_response.content:
        print('Response content:', api_response.content.decode())